# Country Maps

In [9]:
import plotly.plotly as py
import plotly.offline as offline
from plotly.graph_objs import *

import pandas as pd
import os.path

import geocoder
from geopy.distance import vincenty as geodistance

import oauth2client

import gspread
from oauth2client.service_account import ServiceAccountCredentials

Provide parameters

In [11]:
WorkbookTitle = "Azja 2017"
SheetName = "tajwan"

BlogColours = ["#15A9DB","#CCC","#999", "#FFF", "#C7254E", "#073642"]

Ratio = 16.0/9.0
adj = 450.0/430.0
Ratio = Ratio / adj
iw = 800
ih = 450

path = os.path.expanduser("~/Downloads/")

Read spreadsheet

In [12]:
# use creds to create a client to interact with the Google Drive API
scope = ['https://spreadsheets.google.com/feeds']
creds = ServiceAccountCredentials.from_json_keyfile_name('client_secret.json', scope)
client = gspread.authorize(creds)
 
# open sheet and save as dataframe
sheet = client.open(WorkbookTitle).worksheet(SheetName)
df = pd.DataFrame(sheet.get_all_records())
df = df[df['Return'] >= 0]
df = df[df['Arrival'] != ""]

In [13]:
df

,Arrival,ArrivalDate,ArrivalName,Category,CategoryName,Departure,DepartureDate,DepartureName,LeaveDate,Length,Link,Return,TransportToArrival
0,Taipei,2017-08-28,Tajpej,Taiwan,Tajwan,,,,2017-09-09,13,http://jade-sama.github.io/wyprawy/azja-2017/t...,0,Plane
1,Taichung,2017-09-09,Taizhong,Taiwan,Tajwan,,,,2017-09-09,1,http://jade-sama.github.io/wyprawy/azja-2017/t...,0,Train
2,Sun Moon Lake,2017-09-09,Jezioro Slonca i Ksiezyca,Taiwan,Tajwan,,,,2017-09-12,4,http://jade-sama.github.io/wyprawy/azja-2017/t...,1,Bus
4,Tainan,2017-09-12,Tainan,Taiwan,Tajwan,,,,2017-09-15,4,http://jade-sama.github.io/wyprawy/azja-2017/t...,0,Train
5,Hualien,2017-09-15,Hualian,Taiwan,Tajwan,,,,2017-09-15,1,http://jade-sama.github.io/wyprawy/azja-2017/t...,0,Train
6,Taroko National Park,2017-09-15,Park Narodowy Taroko,Taiwan,Tajwan,,,,2017-09-15,1,http://jade-sama.github.io/wyprawy/azja-2017/t...,1,Bus
8,Taitung,2017-09-18,Taidong,Taiwan,Tajwan,,,,2017-09-21,4,http://jade-sama.github.io/wyprawy/azja-2017/t...,0,Train
9,Tainan,2017-09-21,Tainan,Taiwan,Tajwan,,,,2017-09-21,1,http://jade-sama.github.io/wyprawy/azja-2017/t...,0,Train
10,Ananda Suruci,2017-09-21,Farma Ananda Suruci,Taiwan,Tajwan,,,,2017-10-12,22,http://jade-sama.github.io/wyprawy/azja-2017/t...,1,Bus
12,Kaohsiung International Airport,2017-10-12,Port Lotniczy Kaohsiung,Taiwan,Tajwan,,,,2017-10-12,1,http://jade-sama.github.io/wyprawy/azja-2017/t...,0,Train


Generate Map

In [14]:
def create_map(country_name, city_lat, city_lon, min_lon, min_lat, max_lon, max_lat):
    country = [
        Choropleth(
            locationmode = 'country names',
            locations = [country_name],
            z = [1],
            colorscale = [[0,BlogColours[2]],[1, BlogColours[2]]],
            autocolorscale = False,
            showscale = False,
            marker = dict(line = dict(width = 0))
        )
    ]

    city = [
        Scattergeo(
            lat = [city_lat],
            lon = [city_lon],
            mode = 'markers',
            marker = dict(
                size = 28,
                opacity = 1,
                color = BlogColours[0],
                line = dict(width = 1, color = BlogColours[3]))
            )
    ]

    layout = Layout(
        geo = dict(
            projection = dict(type = "equirectangular"),
            lataxis = dict(range = [min_lat, max_lat]),
            lonaxis = dict(range = [min_lon, max_lon]),
            showframe = False,
            showland = True,
            showcountries = False,
            showlakes = False,
            showcoastlines = False,
            landcolor = BlogColours[1],
            bgcolor = BlogColours[3]
        ),
        autosize = True,
        margin = dict(l = 0, r = 0, t = 0, b = 0, pad = 0),
        titlefont = dict (size = 1),
    )

    return dict(data=country+city, layout=layout)


Scale map boundries so the map has the right height/wight ratio (projection adj not included yet)

In [15]:
def scale_map_boundries (min_lon, min_lat, max_lon, max_lat):

    diff_lat = abs(max_lat-min_lat)
    diff_lon = abs(max_lon-min_lon)
    avg_lat = (max_lat+min_lat)/2
    avg_lon = (max_lon+min_lon)/2

    diff_lat_km = geodistance((min_lat,avg_lon), (max_lat,avg_lon)).km
    diff_lon_km = geodistance((avg_lat,min_lon), (avg_lat,max_lon)).km 
    
    if diff_lon_km/diff_lat_km > Ratio:
        min_lat=avg_lat-(diff_lon_km/diff_lat_km)/Ratio*diff_lat/2
        max_lat=avg_lat+(diff_lon_km/diff_lat_km)/Ratio*diff_lat/2

    elif diff_lon_km/diff_lat_km < Ratio:
        min_lon=avg_lon-Ratio/(diff_lon_km/diff_lat_km)*diff_lon/2
        max_lon=avg_lon+Ratio/(diff_lon_km/diff_lat_km)*diff_lon/2

    return [min_lon, min_lat, max_lon, max_lat]

Iterate through rows in dataframe and save each map

In [36]:
country = geocoder.google(df.iloc[0]['Category'])
map_boundries = country.geojson['features'][0]['bbox']
map_boundries = scale_map_boundries(map_boundries[0],map_boundries[1],map_boundries[2],map_boundries[3])

In [37]:
 for i, row in df.iterrows():
        
    fname = row['ArrivalName'].replace(" ", "-").lower()
    
    if not(os.path.isfile(path + fname + ".png")):
    
        print path + fname + ".png"

        city = geocoder.google(row['Arrival'])
        country = geocoder.google(row['Category'])

        fig = create_map(row['Category'], city.latlng[0], city.latlng[1],
                         map_boundries[0],map_boundries[1],map_boundries[2],map_boundries[3])
        
        offline.init_notebook_mode()
        offline.plot(fig,
                 filename = fname + ".html",
                 image='png',
                 image_filename = fname,
                 image_height = ih,
                 image_width = iw,
                 #auto_open = False
        )


C:\Users\Dorota/Downloads/tajpej.png


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.


C:\Users\Dorota/Downloads/taizhong.png


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.


C:\Users\Dorota/Downloads/jezioro-slonca-i-ksiezyca.png


...

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.


C:\Users\Dorota/Downloads/farma-ananda-suruci.png


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.


C:\Users\Dorota/Downloads/port-lotniczy-kaohsiung.png


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.
